# Tag to animal frame: re-orienting tag to match animal's axes

Here's a tutorial my friend Max developed to help with this headache: https://flukeandfeather.com/posts/2024-08-30-animal-orientation-with-imu-ta/

## Load and inspect data
Load pickle file and inspect contents

In [ ]:
import os
import pickle
import pandas as pd

# Import necessary pyologger utilities
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.process_data.sampling import upsample
from pyologger.calibrate_data.tag2animal import *
from pyologger.calibrate_data.zoc import *
from pyologger.plot_data.plotter import plot_depth_correction

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")
color_mapping_path = os.path.join(root_dir, "color_mappings.json")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
deployment_db = metadata.get_metadata("deployment_DB")
logger_db = metadata.get_metadata("logger_DB")
recording_db = metadata.get_metadata("recording_DB")
animal_db = metadata.get_metadata("animal_DB")

# Assuming you have the metadata and deployment_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(deployment_db, data_dir)

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.logger_info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

In [ ]:
data_pkl.event_data

In [ ]:
acc_df = data_pkl.derived_data['calibrated_acc']
mag_df = data_pkl.derived_data['calibrated_mag']
gyr_df = data_pkl.sensor_data['gyroscope']

# Calculate and print sampling frequency for each dataframe
acc_fs = calculate_sampling_frequency(acc_df['datetime'])
print(f"Accelerometer Sampling frequency: {acc_fs} Hz")

mag_fs = calculate_sampling_frequency(mag_df['datetime'])
print(f"Magnetometer Sampling frequency: {mag_fs} Hz")

gyr_fs = calculate_sampling_frequency(gyr_df['datetime'])
print(f"Gyroscope Sampling frequency: {gyr_fs} Hz")

In [ ]:
acc_df

In [ ]:
# acc_data = data_pkl.sensor_data['accelerometer'][['ax','ay','az']]
# mag_data = data_pkl.sensor_data['magnetometer'][['mx', 'my', 'mz']]

acc_data = acc_df[['ax','ay','az']]
mag_data = mag_df[['mx','my','mz']]
gyr_data = gyr_df[['gx', 'gy', 'gz']]

upsampled_columns = []
for col in gyr_data.columns:
    upsampled_col = upsample(gyr_data[col].values, acc_fs / gyr_fs, len(acc_data))  # Apply upsample to each column
    upsampled_columns.append(upsampled_col)  # Append the upsampled column to the list

# Combine the upsampled columns back into a NumPy array
gyr_data_upsampled = np.column_stack(upsampled_columns)
gyr_data = gyr_data_upsampled

acc_data = acc_data.values
mag_data = mag_data.values

# Assuming acc_data and mag_data_upsampled are NumPy arrays
print(f"Gyroscope upsampled to match accelerometer length: acc_data shape= {acc_data.shape}, upsampled gyr_data shape = {gyr_data.shape}")
sampling_rate = acc_fs

In [ ]:
print(data_pkl.deployment_info["Deployment Latitude"])

In [ ]:
?orientation_and_heading_correction

In [ ]:
import pandas as pd

# Define the start and end datetimes with the correct timezone
timezone = acc_df['datetime'].dt.tz  # Get the timezone of the datetime column

# Find start and end time of calm period that tag is near horizontal
start_time = pd.Timestamp("2019-11-08 09:18:33").tz_localize(timezone) 
end_time = pd.Timestamp("2019-11-08 09:18:58").tz_localize(timezone)

# Filter the DataFrame to include only rows within the specified time range
accelerometer_df = data_pkl.sensor_data['accelerometer']
filtered_df = accelerometer_df[(accelerometer_df['datetime'] >= start_time) & (accelerometer_df['datetime'] <= end_time)]

# Calculate the mean of ax, ay, and az columns within the filtered time range
mean_values = filtered_df[['ax', 'ay', 'az']].mean()

# Display the results
print("Average values between 09:18:33 and 09:18:58 on Nov 8, 2019:")
print(mean_values)


In [39]:
abar0 = [-2.124303, -0.699504, -10.225013]
# abar0 = [0,0, -9.8]
deploy_latitude = data_pkl.deployment_info["Deployment Latitude"]
deploy_longitude = data_pkl.deployment_info["Deployment Longitude"]

# latitude= 32.764655 # Seaworld
# longitude= -117.228585 # Seaworld
# abar0 = [0, 0, -9.8] # FOR KILLER WHALE

In [ ]:
# Use the function to get corrected orientation and heading for the entire dataset
pitch_deg, roll_deg, heading_deg, corrected_acc, corrected_mag, corrected_gyr = orientation_and_heading_correction(
    abar0, 
    latitude= deploy_latitude,
    longitude= deploy_longitude,
    acc_data=acc_data, 
    mag_data=mag_data, 
    gyr_data=gyr_data)

In [41]:
# One datetime column from highest sampled data that was matched by other sensors
datetime_data = data_pkl.sensor_data['accelerometer']['datetime']

# Step 1: Create a DataFrame for pitch, roll, and heading
prh_df = pd.DataFrame({
    'datetime': datetime_data,
    'pitch': pitch_deg,
    'roll': roll_deg,
    'heading': heading_deg
})

# Store the 'prh' variable in derived_data
data_pkl.derived_data['prh'] = prh_df
data_pkl.derived_info['prh'] = {
    "channels": ["pitch", "roll", "heading"],
    "metadata": {
        'pitch': {'original_name': 'Pitch (degrees)',
                  'unit': 'degrees',
                  'sensor': 'accelerometer'},
        'roll': {'original_name': 'Roll (degrees)',
                 'unit': 'degrees',
                 'sensor': 'accelerometer'},
        'heading': {'original_name': 'Heading (degrees)',
                    'unit': 'degrees',
                    'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["accelerometer", "magnetometer"],
    "transformation_log": ["calculated_pitch_roll_heading"]
}

# Step 2: Create DataFrames for corrected accelerometer, magnetometer, and gyroscope data
corrected_acc_df = pd.DataFrame({
    'datetime': datetime_data,
    'ax': corrected_acc[:, 0],
    'ay': corrected_acc[:, 1],
    'az': corrected_acc[:, 2]
})

corrected_mag_df = pd.DataFrame({
    'datetime': datetime_data,
    'mx': corrected_mag[:, 0],
    'my': corrected_mag[:, 1],
    'mz': corrected_mag[:, 2]
})

corrected_gyr_df = pd.DataFrame({
    'datetime': datetime_data,
    'gx': corrected_gyr[:, 0],
    'gy': corrected_gyr[:, 1],
    'gz': corrected_gyr[:, 2]
})

# Step 3: Store the corrected accelerometer, magnetometer, and gyroscope data into derived_data
data_pkl.derived_data['corrected_acc'] = corrected_acc_df
data_pkl.derived_info['corrected_acc'] = {
    "channels": ["ax", "ay", "az"],
    "metadata": {
        'ax': {'original_name': 'Acceleration X (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'ay': {'original_name': 'Acceleration Y (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'},
        'az': {'original_name': 'Acceleration Z (m/s^2)',
               'unit': 'm/s^2',
               'sensor': 'accelerometer'}
    },
    "derived_from_sensors": ["accelerometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_mag'] = corrected_mag_df
data_pkl.derived_info['corrected_mag'] = {
    "channels": ["mx", "my", "mz"],
    "metadata": {
        'mx': {'original_name': 'Magnetometer X (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'my': {'original_name': 'Magnetometer Y (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'},
        'mz': {'original_name': 'Magnetometer Z (µT)',
               'unit': 'µT',
               'sensor': 'magnetometer'}
    },
    "derived_from_sensors": ["magnetometer"],
    "transformation_log": ["corrected_orientation"]
}

data_pkl.derived_data['corrected_gyr'] = corrected_gyr_df
data_pkl.derived_info['corrected_gyr'] = {
    "channels": ["gx", "gy", "gz"],
    "metadata": {
        'gx': {'original_name': 'Gyroscope X (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gy': {'original_name': 'Gyroscope Y (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'},
        'gz': {'original_name': 'Gyroscope Z (deg/s)',
               'unit': 'deg/s',
               'sensor': 'gyroscope'}
    },
    "derived_from_sensors": ["gyroscope"],
    "transformation_log": ["corrected_orientation"]
}


In [ ]:
# OVERLAP_START_TIME = '2019-11-08 09:00:00'
# OVERLAP_END_TIME = '2019-11-08 11:30:00'
# ZOOM_START_TIME = '2019-11-08 09:15:00'
# ZOOM_END_TIME = '2019-11-08 09:30:00'


OVERLAP_START_TIME = '2024-01-16 09:30:00'  # Start time for plotting
OVERLAP_END_TIME = '2024-01-16 10:30:00'  # End time for plotting
ZOOM_START_TIME = '2024-01-16 10:00:00'  # Start time for zooming
ZOOM_END_TIME = '2024-01-16 10:02:30'  # End time for zooming
TARGET_SAMPLING_RATE = 10

notes_to_plot = {
    'heartbeat_manual_ok': {'sensor': 'ecg', 'symbol': 'triangle-down', 'color': 'blue'},
    'heartbeat_auto_detect_accepted': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'green'},
    'heartbeat_auto_detect_rejected': {'sensor': 'ecg', 'symbol': 'triangle-up', 'color': 'red'}
}

fig = plot_tag_data_interactive5(
    data_pkl=data_pkl,
    sensors=['ecg', 'accelerometer', 'magnetometer'],
    derived_data_signals=['depth', 'corrected_acc', 'corrected_mag', 'prh'],
    channels={},
    time_range=(OVERLAP_START_TIME, OVERLAP_END_TIME),
    note_annotations=notes_to_plot,
    color_mapping_path=color_mapping_path,
    target_sampling_rate=TARGET_SAMPLING_RATE,
    zoom_start_time=ZOOM_START_TIME,
    zoom_end_time=ZOOM_END_TIME,
    zoom_range_selector_channel='depth',
    plot_event_values=[],
)

In [ ]:
?DataReader.save_to_netcdf

In [ ]:
data_pkl.save_to_netcdf('data/2019-11-08_apfo-001a/outputs/deployment_data_processed.nc')

In [10]:
# Optional: save new pickle file
with open(pkl_path, 'wb') as file:
        pickle.dump(data_pkl, file)